In [17]:
import torch
from torch.nn import Linear
from torch_geometric.nn import ChebConv
import torchvision
import pygsp
import networkx as nx 
from torch_geometric.data import Data
from torch_geometric.utils import from_networkx
from pygsp.graphs.nngraphs.spherehealpix import SphereHealpix
from torch_geometric.nn import knn_graph
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.neighbors import BallTree
import networkx as nx
import yaml
from healpix_pool_unpool import Healpix
import math

torch.manual_seed(1234)

In [2]:
#gra=from_networkx(gra)
data=np.load("processed_images/00000001.npy")
data=torchvision.transforms.functional.to_tensor(data)
data.shape

torch.Size([1, 3145728, 3])

In [55]:
class chebchevconvolution(torch.nn.Module):
    def __init__(self):
        super(chebchevconvolution, self).__init__()
        self.edge_index=[]
        self.weight=[]
        self.conv_layer=[]
        self.model_parameters()
        self.create_architecture()
        
    def create_architecture(self):
        for i in range(self.depth):
            if (i==0):
                self.conv=ChebConv(self.input_channels_convolutionlayer,self.output_channels_convolutionlayer[i],self.kernel_size)
                self.conv_layer.append(self.conv)
            else:
                self.conv=ChebConv(self.output_channels_convolutionlayer[i-1],self.output_channels_convolutionlayer[i],self.kernel_size)
                self.conv_layer.append(self.conv)
            subdivisions = int(self.healpix_resolution_calculator(self.n_pixels)/2**i)
            Graph_pygsp = SphereHealpix(subdivisions, nest=True, k=20)
            graph_netx=Graph_pygsp.to_networkx()
            graph_pyg=from_networkx(graph_netx)
            edge_idx=graph_pyg.edge_index
            self.edge_index.append(edge_idx)
            weight=graph_pyg.weight
            self.weight.append(weight)
        self.pooling_class=Healpix()
            
            
    def healpix_resolution_calculator(self,nodes):
        """Calculate the resolution of a healpix graph
        for a given number of nodes.
        Args:
            nodes (int): number of nodes in healpix sampling
        Returns:
            int: resolution for the matching healpix graph
        """ 
        resolution = int(math.sqrt(nodes / 12))
        return resolution
        
    def model_parameters(self):
        filename="config.yml"
        output_channels_convolutionlayer=[]
        with open(filename) as f:
            my_dict = yaml.safe_load(f)
            self.n_pixels=my_dict["IMAGE PARAMS"]["n_pixels"]
            self.depth=my_dict["IMAGE PARAMS"]["depth"]
            self.kernel_size=my_dict["MODEL PARAMS"]["kernel_size"]
            self.input_channels_convolutionlayer=my_dict["MODEL PARAMS"]["input_channels"]
            self.output_channels_convolutionlayer=my_dict["MODEL PARAMS"]["output_channels"]
            assert (self.depth==len(self.output_channels_convolutionlayer)),"Please check the config.yml file. The depth (under image params) and len(output channel) (under Model params) should be equal"

        

    def forward(self, x):
        for i in range(self.depth):
            if i==0:
                self.h = self.conv_layer[i](x, self.edge_index[i],self.weight[i])
            else:
                self.h=self.conv_layer[i](self.h, self.edge_index[i],self.weight[i])
            self.h =self.pooling_class.pooling(self.h)
            
        
          # Final GNN embedding space.
        
        # Apply a final (linear) classifier.


        return self.h
model = chebchevconvolution()
print(model)

chebchevconvolution(
  (conv): ChebConv(4, 2, K=3, normalization=sym)
)


In [56]:
h = model(data)

In [47]:
model.weight

[tensor([0.4926, 0.4926, 0.3047,  ..., 0.3047, 0.4926, 0.4926]),
 tensor([0.4929, 0.4929, 0.3028,  ..., 0.3028, 0.4929, 0.4929]),
 tensor([0.4939, 0.4939, 0.2951,  ..., 0.2951, 0.4939, 0.4939])]

In [57]:
h

tensor([[[-0.7696, -1.4220],
         [-0.7985, -1.4004],
         [-0.7681, -1.3793],
         ...,
         [ 0.0000,  0.0000],
         [ 0.0000,  0.0000],
         [ 0.0000,  0.0000]]], grad_fn=<PermuteBackward0>)

In [58]:
h.shape

torch.Size([1, 49152, 2])

In [ ]:
h[0,31457]

In [60]:
import healpy as hp
hp.nside2npix(512/8)

49152.0

In [ ]:
h.indices